In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
print('tf版本：',tf.__version__)
# 检查是否存在可用的GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 仅在需要的时候申请显存
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf版本： 2.2.0


In [2]:
! nvidia-smi

Mon May 17 19:49:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   36C    P0    34W / 250W |    455MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
print(tf.config.list_physical_devices('CPU'))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### 一、张量的操作

- tensor定义为张量，这是最基本的数据结构。可以看成是n维array或者list。
- 在tensorflow中的各部分流动传递的都是tensor。
- 编写TensorFlow程序时，操作并传递的基本对象就是tf.tensor。
- 需要定义两个内容，一是数据类型可以是float32，int32,string，二是shape维度

In [12]:
# 0阶张量
mammal = tf.Variable('test',tf.string)
tf.print(tf.rank(mammal)) # 显示阶数
tf.print(tf.shape(mammal)) # 显示张量的结构

0
[]


In [61]:
(tf.rank(mammal))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [17]:
d2_var = tf.random.uniform(shape = (2,4,3)) # 注意最后一维度是张量的最终构成维度

In [18]:
d2_var

<tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
array([[[0.25548053, 0.96570015, 0.834877  ],
        [0.60000885, 0.32667875, 0.5130936 ],
        [0.20464921, 0.95993865, 0.08188474],
        [0.65359795, 0.0105046 , 0.54304445]],

       [[0.4599303 , 0.22353292, 0.8064929 ],
        [0.28975773, 0.8231592 , 0.12648726],
        [0.349697  , 0.2281251 , 0.2587855 ],
        [0.76335585, 0.867838  , 0.01459694]]], dtype=float32)>

In [7]:
d2_var.shape

TensorShape([2, 4, 2])

In [5]:
d3_var1 = tf.random.uniform(shape = (2,3))
tf.print(tf.rank(d3_var1))

2


In [64]:
tf.print(tf.rank(d2_var))
tf.print(tf.shape(d2_var))

3
[2 4 2]


In [19]:
print(tf.rank(d2_var)) # 第一个是rank，表示是几阶，第二个是维度，第三个是数据类型

tf.Tensor(3, shape=(), dtype=int32)


In [66]:
# 1阶张量
mystr=tf.Variable(['hello'],tf.string)
tf.print(tf.rank(mystr))
tf.print(tf.shape(mystr))

1
[1]


In [67]:
# 2阶张量
mymat = tf.Variable([[7],[10]],tf.int16)
tf.print(tf.rank(mymat))
tf.print(tf.shape(mymat))

2
[2 1]


In [68]:
mymat.get_shape()

TensorShape([2, 1])

In [69]:
# 新的创建张量的方法
tf.constant([1,2,3],dtype=tf.int16)

<tf.Tensor: shape=(3,), dtype=int16, numpy=array([1, 2, 3], dtype=int16)>

![张量的基础操作](./markdown_pics/张量的基础操作.png)

tf.random() # 随机初始化，在神经网络的迭代初始值非常常用

tf.feature_column # 特征工程接口

#### 张量的操作 -- tf.string

In [70]:
tf.strings.bytes_split('hello world') # 切割字符串

<tf.Tensor: shape=(11,), dtype=string, numpy=
array([b'h', b'e', b'l', b'l', b'o', b' ', b'w', b'o', b'r', b'l', b'd'],
      dtype=object)>

In [71]:
tf.strings.split('hello world') # 单词切割

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'hello', b'world'], dtype=object)>

In [72]:
help(tf.strings.split) # 同样可以通过help使用文档

Help on function string_split_v2 in module tensorflow.python.ops.ragged.ragged_string_ops:

string_split_v2(input, sep=None, maxsplit=-1, name=None)
    Split elements of `input` based on `sep` into a `RaggedTensor`.
    
    Let N be the size of `input` (typically N will be the batch size). Split each
    element of `input` based on `sep` and return a `RaggedTensor` containing the 
    split tokens. Empty tokens are ignored.
    
    Example:
    
    >>> tf.strings.split('hello world').numpy()
     array([b'hello', b'world'], dtype=object)
    >>> tf.strings.split(['hello world', 'a b c'])
    <tf.RaggedTensor [[b'hello', b'world'], [b'a', b'b', b'c']]>
    
    If `sep` is given, consecutive delimiters are not grouped together and are
    deemed to delimit empty strings. For example, `input` of `"1<>2<><>3"` and
    `sep` of `"<>"` returns `["1", "2", "", "3"]`. If `sep` is None or an empty
    string, consecutive whitespace are regarded as a single separator, and the
    result wil

与tf.string_to_hash_bucket_fast函数的作用一样,tf.string_to_hash_bucket_strong函数也是通过多个 bucket 将输入张量中的每个字符串转换为哈希模式.
哈希函数对进程内字符串的内容是确定性的.散列函数是一个密钥散列函数,其中属性 key 定义散列函数的密钥.key 是 2 个元素的数组.

当输入可能是恶意的时,强大的散列很重要,例如带有附加组件的 URL.攻击者可以尝试将他们的输入散列到同一个 bucket 中以进行拒绝服务攻击或扭曲结果.如果不是不可行的话,强大的散列通过使其难以计算散列到相同 bucket 的输入来防止这种情况.这比tf.string_to_hash_bucket_fast函数的计算时间要多出大约4倍.


In [73]:
tf.strings.to_hash_bucket(['hello','world'],num_buckets = 10) # 转换为hash映射

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([8, 1])>

In [74]:
tf.strings.to_hash_bucket(['hello','world'],num_buckets = 4) # 转换为hash

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1])>

#### 张量的操作 -- tf.debugging

In [75]:
# 自带debug函数
a = tf.random.uniform((10,10))
tf.debugging.assert_equal(x=a.shape,y=(10,10))

In [76]:
# 可以用来断言判断是否错误
tf.debugging.assert_equal(x=a.shape,y=(10,20))

InvalidArgumentError: Condition x == y did not hold.
Indices of first 1 different values:
[[1]]
Corresponding x values:
[10]
Corresponding y values:
[20]
First 2 elements of x:
[10 10]
First 2 elements of y:
[10 20]

#### 张量的操作 -- tf.random

In [11]:
a = tf.random.uniform(shape=(10,5),minval=0,maxval=10, dtype=tf.float16)

In [12]:
a

<tf.Tensor: shape=(10, 5), dtype=float16, numpy=
array([[8.06   , 7.754  , 4.062  , 0.9766 , 3.193  ],
       [9.766  , 2.88   , 6.18   , 2.05   , 0.4492 ],
       [7.703  , 0.713  , 2.305  , 2.607  , 4.844  ],
       [5.203  , 7.08   , 2.852  , 9.47   , 2.94   ],
       [3.799  , 0.4688 , 0.7812 , 6.777  , 0.06836],
       [9.38   , 0.1562 , 8.33   , 4.18   , 5.008  ],
       [8.72   , 9.56   , 8.16   , 7.14   , 1.055  ],
       [6.258  , 5.89   , 4.875  , 0.2148 , 6.21   ],
       [3.447  , 1.885  , 3.574  , 5.664  , 5.617  ],
       [9.97   , 2.568  , 7.273  , 5.46   , 7.18   ]], dtype=float16)>

#### 张量的操作 -- tf.math

In [79]:
a = tf.constant([[1,2],[3,4]])
b = tf.constant([[5,6],[7,8]])

In [80]:
tf.print(tf.math.add(a,b))

[[6 8]
 [10 12]]


In [81]:
tf.print(tf.math.subtract(a,b))

[[-4 -4]
 [-4 -4]]


In [82]:
print(tf.math.subtract(a,b)) # 注意与tf.print的区别

tf.Tensor(
[[-4 -4]
 [-4 -4]], shape=(2, 2), dtype=int32)


In [83]:
tf.print(tf.math.multiply(a,b))

[[5 12]
 [21 32]]


In [84]:
tf.print(tf.math.divide(a,b))

[[0.2 0.33333333333333331]
 [0.42857142857142855 0.5]]


#### 张量的操作 -- tf.dtypes

In [85]:
x= tf.constant([1,8,2,2],dtype=tf.float32)

In [86]:
x

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 8., 2., 2.], dtype=float32)>

In [87]:
x1 = tf.dtypes.cast(x,tf.int32) # 数据类型转换

In [88]:
x1

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 8, 2, 2], dtype=int32)>

### 二、常用层

In [89]:
#常用的两个调用的函数接口一
tf.keras.layers

<module 'tensorflow.keras.layers' from '/home/hp/.local/lib/python3.8/site-packages/tensorflow/keras/layers/__init__.py'>

In [90]:
#常用的两个调用的函数接口二.更加偏底层的调用
tf.nn

<module 'tensorflow._api.v2.nn' from '/home/hp/.local/lib/python3.8/site-packages/tensorflow/_api/v2/nn/__init__.py'>

In [91]:
tf.keras.layers.LSTM

tensorflow.python.keras.layers.recurrent_v2.LSTM

In [92]:
tf.keras.layers.LSTM(100)

In [93]:
tf.keras.layers.Dense(10) # 全连接神经网络

In [94]:
tf.keras.layers.Dense(64,kernel_regularizer=tf.keras.regularizers.l1(0.01)) # L1正则化

- tf.nn.softmax(b) # 激活函数
- tf.keras.layers.Dense(64,activation=tf.nn.relu)

##### 定义一个张量
- a = tf.random.uniform(shape=(10,100,50),minval=0.5,maxval=0.5)

##### 添加一个LSTM层
- x = tf.keras.layers.LSTM(100)(a)
##### 添加一个全连接层
- x = tf.keras.layers.Dense(10)(x)
##### 激活函数激活
- x = tf.nn.softmax(x)

In [19]:
x = tf.random.normal((2,3,2),dtype=tf.float16)
x

<tf.Tensor: shape=(2, 3, 2), dtype=float16, numpy=
array([[[ 0.291  ,  0.6455 ],
        [-0.2448 , -0.2505 ],
        [ 0.1952 , -0.1853 ]],

       [[-0.599  ,  0.4023 ],
        [-1.411  ,  1.978  ],
        [ 0.06213,  0.509  ]]], dtype=float16)>

In [20]:
tf.keras.layers.LSTM(3)(x)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.00591618, -0.01235119, -0.00232592],
       [-0.20553596, -0.09115572,  0.1536798 ]], dtype=float32)>

In [95]:
# 层中增加激活函数
tf.keras.layers.Dense(64,activation='relu')

In [96]:
# 添加nn的激活函数
tf.keras.layers.Dense(64,activation=tf.nn.relu)

In [97]:
# 将L1正则化应用于连接层
tf.keras.layers.Dense(64,kernel_regularizer=tf.keras.regularizers.l1(0.01))
# 将L2正则化应用于连接层
tf.keras.layers.Dense(64,kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [98]:
# 内核初始化为随机正交矩阵的线性层
tf.keras.layers.Dense(64,kernel_initializer='orthogonal')

In [99]:
# 偏差向量初始化为2.0的线性层
tf.keras.layers.Dense(64,bias_initializer=tf.keras.initializers.Constant(2.0))

#### tf.nn是底层函数库，其他各种库都是基于这个底层来做的扩展

#### tf.keras.layers就是基于tf.nn的高度封装

### 三、三种建模方式

#### 1.Sequential顺序模型（一层一层搭积木）

#### 第一种Sequential model

In [9]:
from tensorflow .keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer = tf.keras.optimizers.Adam(0.01),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy']
             )

In [10]:
inputs = tf.random.normal((1000,32))
outputs = tf.random.normal((1000,10))

In [11]:
model.fit(inputs,outputs,epochs=10, batch_size=112,validation_split = 0.5)

Epoch 1/10
5/5 [==============================] - 0s 46ms/step - loss: -0.3640 - accuracy: 0.0880 - val_loss: 0.1352 - val_accuracy: 0.0900
Epoch 2/10
5/5 [==============================] - 0s 10ms/step - loss: -1.5121 - accuracy: 0.1380 - val_loss: 0.0832 - val_accuracy: 0.0820
Epoch 3/10
5/5 [==============================] - 0s 10ms/step - loss: -2.7973 - accuracy: 0.1700 - val_loss: 0.0659 - val_accuracy: 0.0980
Epoch 4/10
5/5 [==============================] - 0s 9ms/step - loss: -4.4811 - accuracy: 0.1900 - val_loss: 0.1557 - val_accuracy: 0.1020
Epoch 5/10
5/5 [==============================] - 0s 10ms/step - loss: -6.2958 - accuracy: 0.2120 - val_loss: 0.0259 - val_accuracy: 0.1100
Epoch 6/10
5/5 [==============================] - 0s 10ms/step - loss: -8.9966 - accuracy: 0.1960 - val_loss: -0.4490 - val_accuracy: 0.1060
Epoch 7/10
5/5 [==============================] - 0s 10ms/step - loss: -13.2347 - accuracy: 0.2080 - val_loss: -0.5549 - val_accuracy: 0.0940
Epoch 8/10
5/5 [==

In [11]:
inputs[0]

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 1.3546274 ,  0.14268984, -0.82131565, -2.2490053 , -0.3050998 ,
       -0.53242284, -1.3360333 ,  0.05588325,  0.10039943,  0.40335533,
       -0.8351888 ,  3.0953043 ,  0.95685333, -1.4780735 ,  0.9717159 ,
       -1.0806279 ,  0.05053141, -0.88527614, -0.04269643, -0.29269946,
       -0.5522828 ,  0.23216873, -0.7380165 ,  1.0796785 , -1.6538912 ,
        0.8465142 ,  0.13871059, -1.245982  ,  0.4553074 , -1.0320964 ,
        1.4239603 , -0.01195545], dtype=float32)>

#### 第二种Sequential model

In [14]:
# 全连接层
model = tf.keras.Sequential([
    layers.Dense(64,activation='relu',input_shape=(32,)), # 第一层
    layers.Dense(64,activation='relu'),
    layers.Dense(10)
])

In [15]:
# 编译模型,定义优化方法（如梯度下降等方法），定义损失函数，定义评估函数
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [16]:
import numpy as np 
data = np.random.random((1000,32))
labels = np.random.random((1000,10))

In [17]:
data

array([[0.03406993, 0.62826506, 0.29718233, ..., 0.95847823, 0.45519122,
        0.04929961],
       [0.60053359, 0.29438587, 0.84084505, ..., 0.84374661, 0.74802406,
        0.05078155],
       [0.17961188, 0.99371134, 0.09022789, ..., 0.16485392, 0.66181138,
        0.30368934],
       ...,
       [0.45523184, 0.74073013, 0.66882025, ..., 0.73657764, 0.09655415,
        0.08958944],
       [0.54652545, 0.8845449 , 0.55456877, ..., 0.61262058, 0.00474666,
        0.86908224],
       [0.9444222 , 0.41840558, 0.91330741, ..., 0.06050699, 0.93923661,
        0.92225304]])

In [18]:
labels

array([[0.70943203, 0.98510217, 0.55678263, ..., 0.76087963, 0.19866244,
        0.49545775],
       [0.61153736, 0.34020389, 0.77490579, ..., 0.13970232, 0.05542107,
        0.44067188],
       [0.09332678, 0.27234881, 0.3837453 , ..., 0.88189108, 0.55400144,
        0.3210031 ],
       ...,
       [0.31172456, 0.60827424, 0.09338189, ..., 0.36155217, 0.87420322,
        0.89520658],
       [0.19210531, 0.50656611, 0.80931547, ..., 0.13149716, 0.9587719 ,
        0.17817325],
       [0.6051731 , 0.34220196, 0.04896444, ..., 0.61074939, 0.99944288,
        0.04132132]])

In [19]:
model.fit(data,labels,epochs=10,batch_size=20)

Epoch 1/10
50/50 [==============================] - 0s 3ms/step - loss: 74.4995 - accuracy: 0.0980
Epoch 2/10
50/50 [==============================] - 0s 3ms/step - loss: 719.0380 - accuracy: 0.1010
Epoch 3/10
50/50 [==============================] - 0s 3ms/step - loss: 2857.6736 - accuracy: 0.0960
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 6843.2012 - accuracy: 0.0970
Epoch 5/10
50/50 [==============================] - 0s 3ms/step - loss: 10762.6904 - accuracy: 0.1030
Epoch 6/10
50/50 [==============================] - 0s 3ms/step - loss: 21288.0820 - accuracy: 0.0890
Epoch 7/10
50/50 [==============================] - 0s 3ms/step - loss: 22025.1465 - accuracy: 0.0950
Epoch 8/10
50/50 [==============================] - 0s 3ms/step - loss: 32196.8086 - accuracy: 0.1130
Epoch 9/10
50/50 [==============================] - 0s 3ms/step - loss: 46300.6914 - accuracy: 0.1200
Epoch 10/10
50/50 [==============================] - 0s 3ms/step - loss: 65028.1211 - acc

#### 2.Funcitional函数模型

相比顺序模型，函数模型应用的更复杂的场景如下：

- 多输入模型
- 多输出模型
- 具有共享图层（同一个图层被调用多次）的模型
- 具有非顺序数据流的模型（如 残余连接）

当然，这部分的定义以后，在训练模型阶段和tf.keras.Sequential一致

In [135]:
# 示例1如下，单个输入
inputs = tf.keras.Input(shape=(32,))
x = layers.Dense(64,activation='relu')(inputs)

x = layers.Dense(64,activation='relu')(x)
predictions =layers.Dense(10)(x)

model = tf.keras.Model(inputs=inputs,outputs = predictions)


model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

import numpy as np 
data = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.fit(data,labels,batch_size = 50, epochs = 5) #epochs表示训练分为几个阶段 
# batch_size 对批次进行迭代

Epoch 1/5
20/20 [==============================] - 0s 3ms/step - loss: 12.5360 - accuracy: 0.1160
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 15.1706 - accuracy: 0.0930
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 18.5589 - accuracy: 0.1010
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 23.8588 - accuracy: 0.0970
Epoch 5/5
20/20 [==============================] - 0s 3ms/step - loss: 30.6897 - accuracy: 0.0970


In [136]:
# 示例2如下，多个输入
inputs1 = tf.keras.Input(shape=(32,))
inputs2 = tf.keras.Input(shape=(32,))
x1 = layers.Dense(64,activation='relu')(inputs1)
x2 = layers.Dense(64,activation='relu')(inputs2)

x = tf.concat([x1,x2],axis = -1)
x = layers.Dense(64,activation='relu')(x)
predictions =layers.Dense(10)(x)

model = tf.keras.Model(inputs=[inputs1,inputs2],outputs = predictions)


model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

import numpy as np 
data1 = np.random.random((1000,32))
data2 = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.fit([data1,data2],labels,batch_size = 30, epochs = 5) #epochs表示训练分为几个阶段 
# batch_size 对批次进行迭代


Epoch 1/5
34/34 [==============================] - 0s 4ms/step - loss: 16.0389 - accuracy: 0.0960
Epoch 2/5
34/34 [==============================] - 0s 4ms/step - loss: 36.7155 - accuracy: 0.0970
Epoch 3/5
34/34 [==============================] - 0s 4ms/step - loss: 66.3767 - accuracy: 0.1120
Epoch 4/5
34/34 [==============================] - 0s 4ms/step - loss: 95.9583 - accuracy: 0.1090
Epoch 5/5
34/34 [==============================] - 0s 4ms/step - loss: 132.4470 - accuracy: 0.1080



1.batch_size

深度学习的优化算法，用大白话来说其实主要就是梯度下降算法，而每次的参数权重更新主要有两种方法。

（1）遍历全部数据集算一次损失函数（真实值和预测值之间的差距，预测值是根据参数向量来更新的），然后算函数对各个参数的梯度（因为参数向量变化了，所以梯度也需要重新计算），更新梯度

这种方法每更新一次参数都要把数据集里的所有样本都看一遍，计算量开销大，计算速度慢，不支持在线学习，这称为Batch gradient descent，批梯度下降。

（2）stochastic gradient descent

每看一个数据就算一下损失函数，然后求梯度更新参数，这个称为随机梯度下降。这个方法速度比较快，但是收敛性能不太好，可能在最优点附近晃来晃去，hit不到最优点。两次参数的更新也有可能互相抵消掉，造成目标函数震荡的比较剧烈。

为了克服两种方法的缺点，现在一般采用的是一种折中手段，mini-batch gradient decent，小批的梯度下降，这种方法把数据分为若干个批，按批来更新参数，这样，一个批中的一组数据共同决定了本次梯度的方向，下降起来就不容易跑偏，减少了随机性。另一方面因为批的样本数与整个数据集相比小了很多，计算量也不是很大。

基本上现在的梯度下降都是基于mini-batch的，所以深度学习框架的函数中经常会出现batch_size，就是指这个意思。

2.iterations

iterations（迭代）：每一次迭代都是一次权重更新，每一次权重更新需要batch_size个数据进行Forward运算得到损失函数，再BP算法(反向传播算法)更新参数。1个iteration等于使用batchsize个样本训练一次。

3.epochs

epochs被定义为向前和向后传播中所有批次的单次训练迭代。这意味着1个周期是整个输入数据的单次向前和向后传递。简单说，epochs指的就是训练过程中数据将被“轮”多少次，就这样。

接下来让我们看个例子：

假设训练集有1000个样本，batchsize=10，那么： 

训练完整个样本集需要： 100次iteration，1次epoch。

具体的计算公式为： one epoch = numbers of iterations = N = 训练样本的数量/batch_size

#### 3.subclassing模型（通过类的继承来自定义模型）

- 通过子类化tf.keras.Model和定义自己的前向传播模型来构建完全可以自定义的模型
- 和 eager execution模型相辅相成
- 面向对象编程

In [109]:
# 定义一个类
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes = 10):
        super().__init__(name = 'my_model') # 调用父类tf.keras.Model的方法
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = layers.Dense(32,activation='relu')
        self.dense_2 = layers.Dense(num_classes)
    
    def call(self,inputs):
        # 定义前向传播
        # 使用在init中已经定义的层
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return x

In [111]:
model = MyModel(num_classes=10)

In [112]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

import numpy as np 
data = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.fit(data,labels,batch_size = 32, epochs = 5) #epochs表示训练分为几个阶段 
# batch_size 对批次进行迭代


Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 12.1791 - accuracy: 0.1000
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 13.2472 - accuracy: 0.1050
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 14.8303 - accuracy: 0.1070
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 16.5227 - accuracy: 0.0960
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 18.3942 - accuracy: 0.0920


In [51]:
#### 总结本节课

![总结本节课](./markdown_pics/总结第二节课.png)

#### 完结

## 问题

- 1.epoch等三个概念是如何的？
- 2.案例中的演示中的shape和张量仍然不够直观，和实际的应用场景下不够贴合，不能够将展示的案例更好的融入场景理解。

In [8]:
# 反向传播的案例测试

In [9]:
def error_func(label,pred):
    e_total = ((label - pred)**2)/2
    return e_total

In [13]:
error_func(0.01,0.75136507)

0.27481108350805245

In [14]:
error_func(0.99,0.772928465)

0.02356002565362811

##### super函数的作用 https://blog.csdn.net/qq_38787214/article/details/87902291

In [113]:
class Root(object):
    def __init__(self):
        self.x = '这是属性'

    def fun(self):
        print(self.x)
        print('这是方法')


class A(Root):
    def __init__(self):
        super().__init__()
        print('实例化时执行')


test = A()  # 实例化类
test.fun()  # 调用方法
test.x  # 调用属性a

实例化时执行
这是属性
这是方法


'这是属性'

In [14]:
import numpy as np
-np.log2(6/9)*6/9 -np.log2(3/9)*3/9

0.9182958340544898

##### super的调用测试，方法使用父类方法，但是属性是子类自己的属性

In [48]:
class A:
    def __init__(self):
        self.n = 2

    def add(self, m):
        print('self is {0} @A.add'.format(self))
        print('A1:',self.n)
        self.n += m
        print('A2:',self.n)


class B(A):
    def __init__(self):
        self.n = 3

    def add(self, m):
        print('self is {0} @B.add'.format(self))
        print('B1:',self.n)
        super().add(m)
        self.n += 3
        print('B2:',self.n)

In [51]:
test = B()
test.add(2)
print(test.n)

self is <__main__.B object at 0x7f5ab043c100> @B.add
B1: 3
self is <__main__.B object at 0x7f5ab043c100> @A.add
A1: 3
A2: 5
B2: 8
8


In [63]:
# 面向对象编程最基本的结构就是定义属性和方法
class Dad:
    def __init__(self, content = 'I love my Son and Daughter.'):
        self.content = content # 这是属性
    def say(self):  # 这是方法
        print(f'Dad is Say:{self.content}')

In [69]:
hex(id(Dad().say()))

Dad is Say:I love my Son and Daughter.


'0x7f87ab868200'

In [57]:
class Son(Dad):
    def __init__(self, content = 'I love my Dad and my Sister.'):
        self.content = content
    def say(self):
        super().say()  # 注意此处super会调用父类的方法，但是不会调用父类的属性，仍然会使用自己的属性

In [58]:
class Daughter(Dad):
    def __init__(self, content = 'I love my Dad and my Brother.'):
        self.content = content
    pass  # 此处没有定义之类的say方法，那么一旦该之类需要调用say方法，会自动回溯到子类的上一级，看看父类是否有这个方法，有就调用

In [59]:
def say_twice(test):
    test.say()
    test.say()

In [60]:
# 多态的展示形式，是为了体现，当子类实例化为对象以后，函数会先去调用子类方法，如果没有，则会去调用父类的方法
# 由于我们在super已经定义了son调用的是父类的say方法，所以这里的方法段是父类的方法，但是，属性却会使用自己的属性
# 注意执行的内容并不是我们想要的内容，想要的内容应该是Son is say:I love my Dad and my Sister.以此来达到认识概念目的。
son = Son()
say_twice(son)

Dad is Say:I love my Dad and my Sister.
Dad is Say:I love my Dad and my Sister.


In [61]:
# 我们没有定义子类的say方法，仍然会自动回溯调用父类方法，但是属性仍然是子类自己的属性
# 注意执行的内容并不是我们想要的内容，想要的内容应该是Daughter is say:I love my Dad and my Brother.以此来达到认识概念目的。
dau = Daughter()
say_twice(dau)

Dad is Say:I love my Dad and my Brother.
Dad is Say:I love my Dad and my Brother.
